In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
# print(pd.__version__)
import leaguedata as ld

#Progress bars wooo
from ipywidgets import IntProgress
from IPython.display import display
from IPython.display import Markdown

In [2]:
matches = ld.get_all_matches()
print(len(matches))

7620


In [6]:
# num_games = 50
# data = None
# prev_structures = test.prev_kills(time)
# prev_structures_5_mins = prev_structures.loc[prev_structures['Time'] > (time-5)]
# print(prev_structures)
# print(prev_structures_5_mins)
# print(len(prev_structures_5_mins.index)/5)
# print(test.kill_freq(time))
# print(test.prev_kills(20)[2])
# print(test.game_state(5))

# for num in range(num_games):
#     game = matches[num]
#     for minute in range(1,game.length()):
#         segment_data = game.game_state(minute)
# #         print(segment_data)
#         if minute == 0 and num == 0:
#             data = segment_data
#         else:
#             data = pd.concat([data,segment_data],ignore_index=True)
# print(data)
data = pd.concat([pd.read_csv("final_data_1.csv"),pd.read_csv("final_data_2.csv")],ignore_index=True)
data_no_overlap = data.drop(['Kill freq','Structure freq'], axis=1)
data_no_overlap

,Address,Minute,Blue kills,Blue deaths,Blue kill freq,Blue death freq,Red structures taken,Blue structures lost,Blue structure freq,Red structure freq,Blue gold,Gold diff,Delta gold diff,Blue win
0,http://matchhistory.na.leagueoflegends.com/en/...,1,0,0,0.0,0.0,0,0,0.0,0.0,2415.0,0.0,0.0,True
1,http://matchhistory.na.leagueoflegends.com/en/...,2,0,0,0.0,0.0,0,0,0.0,0.0,2415.0,0.0,0.0,True
2,http://matchhistory.na.leagueoflegends.com/en/...,3,0,0,0.0,0.0,0,0,0.0,0.0,2711.0,-14.0,-14.0,True
3,http://matchhistory.na.leagueoflegends.com/en/...,4,0,0,0.0,0.0,0,0,0.0,0.0,3887.0,-65.0,-65.0,True
4,http://matchhistory.na.leagueoflegends.com/en/...,5,0,0,0.0,0.0,0,0,0.0,0.0,5068.0,-268.0,-268.0,True
5,http://matchhistory.na.leagueoflegends.com/en/...,6,0,0,0.0,0.0,0,0,0.0,0.0,6171.0,-431.0,-417.0,True
6,http://matchhistory.na.leagueoflegends.com/en/...,7,0,0,0.0,0.0,0,0,0.0,0.0,7412.0,-488.0,-423.0,True
7,http://matchhistory.na.leagueoflegends.com/en/...,8,0,0,0.0,0.0,0,0,0.0,0.0,8661.0,-789.0,-521.0,True
8,http://matchhistory.na.leagueoflegends.com/en/...,9,0,0,0.0,0.0,0,0,0.0,0.0,10154.0,-494.0,-63.0,True
9,http://matchhistory.na.leagueoflegends.com/en/...,10,0,0,0.0,0.0,0,0,0.0,0.0,11361.0,-625.0,-137.0,True


In [11]:
#Standardisation for LASSO
datatest = data_no_overlap.drop('Address', axis=1)
means = data_no_overlap.mean(axis=0)
stds = data_no_overlap.std(axis=0)
#print(data)
datatest = (datatest - means) / stds
datatest["Blue win"] = data["Blue win"] # set the wins back to bool
datatest

         Minute  Blue kills  Blue deaths  Blue kill freq  Blue death freq  \
0     -1.549939   -0.930210    -0.889439       -0.794021        -0.769586   
1     -1.465209   -0.930210    -0.889439       -0.794021        -0.769586   
2     -1.380478   -0.930210    -0.889439       -0.794021        -0.769586   
3     -1.295748   -0.930210    -0.889439       -0.794021        -0.769586   
4     -1.211017   -0.930210    -0.889439       -0.794021        -0.769586   
5     -1.126287   -0.930210    -0.889439       -0.794021        -0.769586   
6     -1.041556   -0.930210    -0.889439       -0.794021        -0.769586   
7     -0.956826   -0.930210    -0.889439       -0.794021        -0.769586   
8     -0.872095   -0.930210    -0.889439       -0.794021        -0.769586   
9     -0.787365   -0.930210    -0.889439       -0.794021        -0.769586   
10    -0.702634   -0.720493    -0.889439       -0.234315        -0.769586   
11    -0.617904   -0.720493    -0.889439       -0.234315        -0.769586   

In [72]:
#This step makes it real u feel me? only run once or ur original data from the 20 minute step previously is lost
# data_backup = data
# data = datatest

In [15]:
from sklearn.model_selection import train_test_split

y = datatest['Blue win'].values
X = datatest.drop(['Blue win'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)
print(len(X_test)/len(X), len(X_train)/len(X))

0.20002780738749595 0.7999721926125041


,Minute,Blue kills,Blue deaths,Blue kill freq,Blue death freq,Red structures taken,Blue structures lost,Blue structure freq,Red structure freq,Blue gold,Gold diff,Delta gold diff
10339,0.737784,1.166956,1.401229,-0.794021,-0.204066,0.563394,1.079921,0.160532,0.236271,0.842832,-0.424655,-1.098614
8050,-1.380478,-0.930210,-0.889439,-0.794021,-0.769586,-0.960053,-0.945729,-0.685536,-0.657178,-1.335820,-0.040100,-0.034523
261,-0.617904,-0.510777,-0.889439,-0.234315,-0.769586,-0.960053,-0.540599,-0.685536,0.236271,-0.736285,0.119422,0.428684
13549,-1.380478,-0.930210,-0.889439,-0.794021,-0.769586,-0.960053,-0.945729,-0.685536,-0.657178,-1.330846,-0.021590,0.014371
17658,2.940776,4.102990,3.691898,-0.794021,-0.769586,2.086840,1.079921,-0.685536,-0.657178,3.459104,1.966459,-0.516388
21204,0.398862,-0.091343,1.192987,0.325390,1.492494,-0.198330,0.674791,1.006600,0.236271,0.406067,-0.959965,-0.138745
610,-0.278982,-0.930210,0.151774,-0.794021,1.492494,-0.198330,-0.540599,1.006600,0.236271,-0.493336,-0.455585,-0.699741
7126,0.907245,-0.091343,2.442442,-0.234315,2.623534,0.182532,3.510702,-0.685536,5.596968,0.546732,-3.168177,-1.359168
6084,-0.024790,-0.720493,-0.472954,-0.794021,-0.769586,0.182532,-0.135469,1.852668,1.129721,-0.160462,-0.040830,-0.659853
14973,-0.617904,-0.720493,-0.472954,-0.794021,-0.769586,-0.198330,-0.135469,-0.685536,-0.657178,-0.607475,-0.158705,-0.043530


In [34]:
Penalty = 0.2 #Penalty: 1 small - 0.01 big, from slides
from sklearn.linear_model import LogisticRegression as logreg
# print(titanic[["Pclass_1","Pclass_2","Pclass_3"]])

# y = datatest['Blue Win'].values#.reshape(-1, 1)
# print(y)
# X = datatest.drop('Blue Win', axis=1)

# print(X)
# print(y)
# mylr = logreg(C=Penalty, penalty='l1', solver='liblinear')
mylr = logreg()
mylr.fit(X_train,y_train)

LogisticRegression(C=0.2, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [35]:
from scipy import stats
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score

class ModelSummary:
    """ This class extracts a summary of the model
    
    Methods
    -------
    get_se()
        computes standard error
    get_ci(SE_est)
        computes confidence intervals
    get_pvals()
        computes p-values
    get_summary(name=None)
        prints the summary of the model
    """
    
    def __init__(self, clf, X, y):
        """
        Parameters
        ----------
        clf: class
            the classifier object model
        X: pandas Dataframe
            matrix of predictors
        y: numpy array
            matrix of variable
        """
        self.clf = clf
        self.X = X
        self.y = y
        pass
    
    def get_se(self):
        """Computes the standard error

        Returns
        -------
            numpy array of standard errors
        """
        # from here https://stats.stackexchange.com/questions/89484/how-to-compute-the-standard-errors-of-a-logistic-regressions-coefficients
        predProbs = self.clf.predict_proba(self.X)
        X_design = np.hstack([np.ones((self.X.shape[0], 1)), self.X])
        V = np.diagflat(np.product(predProbs, axis=1))
        covLogit = np.linalg.inv(np.dot(np.dot(X_design.T, V), X_design))
        return np.sqrt(np.diag(covLogit))

    def get_ci(self, SE_est):
        """Computes the confidence interval

        Parameters
        ----------
        SE_est: numpy array
            matrix of standard error estimations
        
        Returns
        -------
        cis: numpy array
            matrix of confidence intervals
        """
        p = 0.975
        df = len(self.X) - 2
        crit_t_value = stats.t.ppf(p, df)
        coefs = np.concatenate([self.clf.intercept_, self.clf.coef_[0]])
        upper = coefs + (crit_t_value * SE_est)
        lower = coefs - (crit_t_value * SE_est)
        cis = np.zeros((len(coefs), 2))
        cis[:,0] = lower
        cis[:,1] = upper
        return cis
    
    def get_pvals(self):
        """Computes the p-value

        Returns
        -------
        p: numpy array
            matrix of p-values
        """
        # from here https://stackoverflow.com/questions/25122999/scikit-learn-how-to-check-coefficients-significance
        p = self.clf.predict_proba(self.X)
        n = len(p)
        m = len(self.clf.coef_[0]) + 1
        coefs = np.concatenate([self.clf.intercept_, self.clf.coef_[0]])
        se = self.get_se()
        t =  coefs/se  
        p = (1 - stats.norm.cdf(abs(t))) * 2
        return p
    
    def get_summary(self, names=None):
        """Prints the summary of the model

        Parameters
        ----------
        names: list
            list of the names of predictors
        """
        ses = self.get_se()
        cis = self.get_ci(ses)
        lower = cis[:, 0]
        upper = cis[:, 1]
        pvals = self.get_pvals()
        coefs = np.concatenate([self.clf.intercept_, self.clf.coef_[0]])
        data = []
        for i in range(len(coefs)):
            currlist = []
            currlist.append(np.round(coefs[i], 3))
            currlist.append(np.round(ses[i], 3))
            currlist.append(np.round(pvals[i], 3))
            currlist.append(np.round(lower[i], 3))
            currlist.append(np.round(upper[i], 3))
            data.append(currlist)
        cols = ['coefficient', 'std', 'p-value', '[0.025', '0.975]']
        sumdf = pd.DataFrame(columns=cols, data=data)
        if names is not None:
            new_names = ['intercept']*(len(names) + 1)
            new_names[1:] = [i for i in names]
            sumdf.index = new_names
        else:
            try:
                names = list(self.X.columns)
                new_names = ['intercept']*(len(names) + 1)
                new_names[1:] = [i for i in names]
                sumdf.index = new_names
            except:
                pass
        print(sumdf)
        acc = accuracy_score(self.y, self.clf.predict(self.X))
        confmat = confusion_matrix(self.y, self.clf.predict(self.X))
        print('-'*60)
        print('Confusion Matrix (total:{}) \t Accuracy: \t  {}'.format(len(self.X),np.round(acc, 3)))
        print('  TP: {} | FN: {}'.format(confmat[1][1],confmat[1][0]))
        print('  FP: {} | TN: {}'.format(confmat[0][1],confmat[0][0]))

In [36]:
modsummary = ModelSummary(mylr, X_train, y_train)
modsummary.get_summary()

                      coefficient    std  p-value  [0.025  0.975]
intercept                   0.075  0.019    0.000   0.037   0.112
Minute                     -0.571  0.179    0.001  -0.922  -0.220
Blue kills                  0.001  0.062    0.985  -0.120   0.122
Blue deaths                -0.372  0.063    0.000  -0.496  -0.249
Blue kill freq              0.357  0.037    0.000   0.285   0.429
Blue death freq            -0.330  0.037    0.000  -0.403  -0.258
Red structures taken       -0.074  0.062    0.235  -0.196   0.048
Blue structures lost        0.006  0.058    0.913  -0.107   0.119
Blue structure freq         0.103  0.036    0.005   0.032   0.175
Red structure freq         -0.111  0.035    0.002  -0.180  -0.043
Blue gold                   0.841  0.207    0.000   0.434   1.247
Gold diff                   1.254  0.066    0.000   1.125   1.383
Delta gold diff             0.025  0.041    0.541  -0.055   0.104
------------------------------------------------------------
Confusion Matri

In [75]:
#testing
import random

# test = matches[12]
# print(random.randint(0,test.length()))
num_test_games = 50
test_data = None
# for num in range(num_games+1,num_test_games+num_games+1):
#     print("test")


for num in range(num_games+1,num_test_games+num_games+1):
    game = matches[num]
    for minute in range(1,game.length()):
        test_segment_data = game.game_state(minute)
#         print(segment_data)
        if minute == 0 and num == num_games+1:
            test_data = test_segment_data
        else:
            test_data = pd.concat([test_data,test_segment_data],ignore_index=True)
# for num in range(num_games+1,num_test_games):
#     game = matches[num]
#     for minute in range(game.length()):
#         test_segment_data = game.game_state(minute)
#         if i == 0 and num == num_games+1:
#             test_data = test_segment_data
#         else:
#             test_data = pd.concat([test_data,test_segment_data],ignore_index=True)

lol = test_data["Blue Win"]
test_data = (test_data - means) / stds
test_data["Blue Win"] = lol
        
print(test_data)
# print(data)

      minute  Blue kills  Blue Deaths  Kill freq  Blue kill freq  \
0          1           0            0        0.0             0.0   
1          2           0            0        0.0             0.0   
2          3           0            0        0.0             0.0   
3          4           0            0        0.0             0.0   
4          5           0            0        0.0             0.0   
...      ...         ...          ...        ...             ...   
1877      30           9            8        0.6             0.4   
1878      31           9            8        0.6             0.4   
1879      32          14           10        2.0             1.4   
1880      33          14           10        1.6             1.2   
1881      34          14           10        1.4             1.0   

      Blue death freq  Red structures taken  Blue structures lost  \
0                 0.0                     0                     0   
1                 0.0                     0  

In [76]:
test_X = test_data.drop('Blue Win', axis=1)
real_y = test_data['Blue Win'].values.reshape(-1, 1)

print(test_X)
print(real_y)


      minute  Blue kills  Blue Deaths  Kill freq  Blue kill freq  \
0          1           0            0        0.0             0.0   
1          2           0            0        0.0             0.0   
2          3           0            0        0.0             0.0   
3          4           0            0        0.0             0.0   
4          5           0            0        0.0             0.0   
...      ...         ...          ...        ...             ...   
1877      30           9            8        0.6             0.4   
1878      31           9            8        0.6             0.4   
1879      32          14           10        2.0             1.4   
1880      33          14           10        1.6             1.2   
1881      34          14           10        1.4             1.0   

      Blue death freq  Red structures taken  Blue structures lost  \
0                 0.0                     0                     0   
1                 0.0                     0  

In [77]:
y_predict = mylr.predict(test_X)
print(y_predict)

[False False False ...  True  True  True]


In [78]:
test_summary = ModelSummary(mylr, test_X, real_y)
test_summary.get_summary()

                      coefficient           std  p-value        [0.025  \
intercept                   0.477  5.920000e-01    0.421 -6.840000e-01   
minute                     -0.985  3.880000e-01    0.011 -1.746000e+00   
Blue kills                  2.903  5.914000e+00    0.624 -8.696000e+00   
Blue Deaths                -3.360  3.856510e+02    0.993 -7.597090e+02   
Kill freq                   0.000           NaN      NaN           NaN   
Blue kill freq             -0.071           NaN      NaN           NaN   
Blue death freq             0.017           NaN      NaN           NaN   
Red structures taken        0.549  9.119230e+02    1.000 -1.787938e+03   
Blue structures lost        1.061  1.510141e+03    0.999 -2.960669e+03   
Structure freq              0.000  4.952396e+08    1.000 -9.712770e+08   
Blue structure freq         0.249  4.952396e+08    1.000 -9.712770e+08   
Red structure freq         -0.374  4.952396e+08    1.000 -9.712770e+08   
Gold diff                   1.470  5.8

<ipython-input-74-336093257511>:49: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(covLogit))
<ipython-input-74-336093257511>:49: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(covLogit))
/Users/Sean/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:1847: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


In [79]:
# print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(test_X,real_y)))
score = 0
for result in range(len(real_y)):
    if real_y[result] == y_predict[result]:
        score += 1
accuracy = score/len(real_y)
print(accuracy)

0.7444208289054197
